pkg-config在编译应用程序和库的时候作为一个工具来使用。
```
gcc -o test test.c `pkg-config --libs --cflags glib-2.0`

```

 
```
# 当前安装的库
pkg-config --list-all

```

pkg-config会到/usr/lib/pkconfig/目录下去寻找glib-2.0.pc文件。 
此时我们需要通过PKG_CONFIG_PATH环境变量来指定pkg-config还应该在哪些地方去寻找.pc文件。

```
export PKG_CONFIG_PATH=$PKG_CONFIG_PATH:/usr/local/lib/pkgconfig/

```

库文件在链接（静态库和共享库）和运行（仅限于使用共享库的程序）时被使用，其搜索路径是在系统中进行设置的。一般Linux系统把/lib和/usr/lib这两个目录作为默认的库搜索路径，所以使用这两个目录中的库时不需要进行设置搜索路径即可直接使用。对于处于默认库搜索路径之外的库，需要将库的位置添加到库的搜索路径之中。设置库文件的搜索路径有下列两种方式，可任选其中一种使用：

- 在环境变量LD_LIBRARY_PATH中指明库的搜索路径
- 在/etc/ld.so.conf文件中添加库的搜索路径  

将自己可能存放库文件的路径都加入到/etc/ld.so.conf中是明智的选择。

需要注意的是：第二种搜索路径的设置方式对于程序链接时的库（包括共享库和静态库）的定位已经足够了。但是对于使用了共享库的程序的执行还是不够的，这是因为为了加快程序执行时对共享库的定位速度，避免使用搜索路径查找共享库的低效率，所以是直接读取库列表文件/etc/ld.so.cache的方式从中进行搜索。/etc/ld.so.cache是一个非文本的数据文件，不能直接编辑，它是根据/etc/ld.so.conf中设置的搜索路径由/sbin/ldconfig命令将这些搜索路径下的共享库文件集中在一起而生成的（ldconfig命令要以root权限执行）。因此为了保证程序执行时对库的定位，在/etc/ld.so.conf中进行了库搜索路径的设置之后，还必须要运行/sbin/ldconfig命令更新/etc/ld.so.cache文件之后才可以。

ldconfig，简单的说，它的作用就是将/etc/ld.so.conf列出的路径下的库文件缓存到/etc/ld.so.cache以供使用。因此当安装完一些库文件（例如刚安装好glib)，或者修改ld.so.conf增加新的库路径之后，需要运行一下/sbin/ldconfig使所有的库文件都被缓存到ld.so.cache中。如果没有这样做，即使库文件明明就在/usr/lib下的，也是不会被使用的，结果在编译过程中报错。

在程序链接时，对于库文件（静态库和共享库）的搜索路径，除了上面的设置方式之外，还可以通过-L参数显示指定。因为用-L设置的路径将被优先搜索，所以在链接的时候通常都会以这种方式直接指定要链接的库的路径。

前面已经说明过了，库搜索路径的设置有两种方式：在环境变量 LD_LIBRARY_PATH 中设置以及在 /etc/ld.so.conf 文件中设置。其中，第二种设置方式需要 root 权限，以改变 /etc/ld.so.conf 文件并执行 /sbin/ldconfig 命令。而且，当系统重新启动后，所有的基于 GTK2 的程序在运行时都将使用新安装的 GTK+ 库。不幸的是，由于 GTK+ 版本的改变，这有时会给应用程序带来兼容性的问题，造成某些程序运行不正常。为了避免出现上面的这些情况，在 GTK+ 及其依赖库的安装过程中对于库的搜索路径的设置将采用第一种方式进行。这种设置方式不需要 root 权限，设置也简单：

```
# export LD_LIBRARY_PATH=/opt/gtk/lib:$LD_LIBRARY_PATH
# echo $LD_LIBRARY_PATH

```

